# Prepare Data - Download and Insert into Database

This notebook downloads the green taxi trip data and taxi zone lookup data, then inserts them into the PostgreSQL database.

In [ ]:
# Import required libraries
import os
import pandas as pd
from sqlalchemy import create_engine
import urllib.request
from pathlib import Path

## Step 1: Download Files

In [ ]:
# URLs for the datasets
parquet_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet"
csv_url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"

# Local file paths
parquet_file = "green_tripdata_2025-11.parquet"
csv_file = "taxi_zone_lookup.csv"

# Download parquet file
print("Downloading green taxi data...")
urllib.request.urlretrieve(parquet_url, parquet_file)
print(f"✓ Downloaded: {parquet_file}")

# Download CSV file
print("Downloading taxi zone lookup data...")
urllib.request.urlretrieve(csv_url, csv_file)
print(f"✓ Downloaded: {csv_file}")

## Step 2: Set up Database Connection

In [ ]:
# Database connection parameters
# Update these with your actual database credentials
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME', 'ny_taxi')
DB_USER = os.getenv('DB_USER', 'postgres')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'postgres')

# Create connection string
connection_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# Create SQLAlchemy engine
engine = create_engine(connection_string)

print("Database connection established")

## Step 3: Read and Insert Green Taxi Data

In [ ]:
# Read parquet file
print("Reading parquet file...")
df_green = pd.read_parquet(parquet_file)
print(f"✓ Loaded {len(df_green):,} rows")
print(f"\nData types:")
print(df_green.dtypes)
print(f"\nFirst few rows:")
df_green.head()

In [ ]:
# Fix datatypes for green taxi data
print("Fixing datatypes...")

# Convert datetime columns (common column names in taxi data)
datetime_columns = []
for col in df_green.columns:
    if 'datetime' in col.lower() or 'date' in col.lower() or 'time' in col.lower():
        datetime_columns.append(col)
        df_green[col] = pd.to_datetime(df_green[col], errors='coerce')
        print(f"  Converted {col} to datetime")

# Convert numeric columns that might be strings
numeric_columns = ['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 
                   'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 
                   'congestion_surcharge', 'ehail_fee']
for col in numeric_columns:
    if col in df_green.columns:
        df_green[col] = pd.to_numeric(df_green[col], errors='coerce')
        print(f"  Converted {col} to numeric")

# Display updated datatypes
print("\nUpdated data types:")
print(df_green.dtypes)
print(f"\nSample of converted data:")
df_green.head()

In [ ]:
# Insert into public.green_taxi_data
# Using if_exists='replace' to replace existing data, change to 'append' if you want to keep existing data
print("Inserting data into public.green_taxi_data...")
df_green.to_sql(name='green_taxi_data', 
                con=engine, 
                schema='public',
                if_exists='replace',  # Change to 'append' if you want to add to existing data
                index=False,
                method='multi',  # Faster insertion
                chunksize=10000)  # Insert in chunks
print("✓ Data inserted successfully into public.green_taxi_data")

## Step 4: Read and Insert Taxi Zone Lookup Data

In [ ]:
# Read CSV file
print("Reading CSV file...")
df_zones = pd.read_csv(csv_file)
print(f"✓ Loaded {len(df_zones):,} rows")
print(f"\nData types:")
print(df_zones.dtypes)
print(f"\nFirst few rows:")
df_zones.head()

In [ ]:
# Fix datatypes for taxi zone lookup data
print("Fixing datatypes...")

# Convert LocationID to integer if it exists
if 'LocationID' in df_zones.columns:
    df_zones['LocationID'] = pd.to_numeric(df_zones['LocationID'], errors='coerce').astype('Int64')
    print(f"  Converted LocationID to integer")

# Ensure string columns are properly typed
string_columns = ['Borough', 'Zone', 'service_zone']
for col in string_columns:
    if col in df_zones.columns:
        df_zones[col] = df_zones[col].astype('string')
        print(f"  Converted {col} to string")

# Display updated datatypes
print("\nUpdated data types:")
print(df_zones.dtypes)
print(f"\nSample of converted data:")
df_zones.head()

In [ ]:
# Insert into public.taxi_zone_lookup
print("Inserting data into public.taxi_zone_lookup...")
df_zones.to_sql(name='taxi_zone_lookup', 
                con=engine, 
                schema='public',
                if_exists='replace',  # Change to 'append' if you want to keep existing data
                index=False)
print("✓ Data inserted successfully into public.taxi_zone_lookup")

## Step 5: Verify Data Insertion

In [ ]:
# Verify green_taxi_data
query_green = "SELECT COUNT(*) as row_count FROM public.green_taxi_data;"
result_green = pd.read_sql(query_green, engine)
print(f"Rows in public.green_taxi_data: {result_green['row_count'].iloc[0]:,}")

# Verify taxi_zone_lookup
query_zones = "SELECT COUNT(*) as row_count FROM public.taxi_zone_lookup;"
result_zones = pd.read_sql(query_zones, engine)
print(f"Rows in public.taxi_zone_lookup: {result_zones['row_count'].iloc[0]:,}")

print("\n✓ Data verification complete!")

## Step 6: Query Analysis

In [ ]:
# Run query and get result as DataFrame
query = """
SELECT COUNT(*) as trips_count
FROM public.green_taxi_data
WHERE lpep_pickup_datetime >= '2025-11-01' 
  AND lpep_pickup_datetime < '2025-12-01'
  AND trip_distance <= 1;
"""

result = pd.read_sql(query, engine)
print(f"Number of trips in November 2025 with distance <= 1 mile: {result['trips_count'].iloc[0]:,}")
result

In [ ]:
# Pickup day with the longest trip distance (trips < 100 miles to exclude data errors)
query_longest = """
SELECT DATE(lpep_pickup_datetime) AS pickup_date,
       MAX(trip_distance) AS max_trip_distance
FROM public.green_taxi_data
WHERE trip_distance < 100
GROUP BY DATE(lpep_pickup_datetime)
ORDER BY max_trip_distance DESC
LIMIT 1;
"""

result_longest = pd.read_sql(query_longest, engine)
pickup_date = result_longest['pickup_date'].iloc[0]
max_dist = result_longest['max_trip_distance'].iloc[0]
print(f"Pickup day with longest trip: {pickup_date} (max distance: {max_dist:.2f} miles)")
result_longest

In [ ]:
# Drop-off zone with largest tip for pickups from "East Harlem North" in November 2025
query_tip = """
SELECT tz_do."Zone" AS dropoff_zone,
       tz_do."Borough" AS dropoff_borough,
       SUM(g.tip_amount) AS total_tips
FROM public.green_taxi_data g
JOIN public.taxi_zone_lookup tz_pu ON g."PULocationID" = tz_pu."LocationID" AND tz_pu."Zone" = 'East Harlem North'
JOIN public.taxi_zone_lookup tz_do ON g."DOLocationID" = tz_do."LocationID"
WHERE g.lpep_pickup_datetime >= '2025-11-01' 
  AND g.lpep_pickup_datetime < '2025-12-01'
GROUP BY tz_do."Zone", tz_do."Borough"
ORDER BY total_tips DESC
LIMIT 1;
"""

result_tip = pd.read_sql(query_tip, engine)
dropoff_zone = result_tip['dropoff_zone'].iloc[0]
dropoff_borough = result_tip['dropoff_borough'].iloc[0]
total_tips = result_tip['total_tips'].iloc[0]
print(f"Drop-off zone with largest tip (pickups from East Harlem North, Nov 2025): {dropoff_zone} ({dropoff_borough}) — ${total_tips:,.2f}")
print(result_tip)

In [ ]:
# Pickup zone with largest total_amount (sum) on November 18th, 2025
query_zone = """
SELECT tz."Zone" AS pickup_zone,
       tz."Borough" AS borough,
       SUM(g.total_amount) AS total_amount_sum
FROM public.green_taxi_data g
JOIN public.taxi_zone_lookup tz ON g."PULocationID" = tz."LocationID"
WHERE g.lpep_pickup_datetime >= '2025-11-18' 
  AND g.lpep_pickup_datetime < '2025-11-19'
GROUP BY tz."Zone", tz."Borough"
ORDER BY total_amount_sum DESC
LIMIT 1;
"""

result_zone = pd.read_sql(query_zone, engine)
pickup_zone = result_zone['pickup_zone'].iloc[0]
borough = result_zone['borough'].iloc[0]
total = result_zone['total_amount_sum'].iloc[0]
print(f"Pickup zone with largest total_amount on 2025-11-18: {pickup_zone} ({borough}) — ${total:,.2f}")
result_zone